# KRX 연간 재무제표 데이터 수집

In [7]:
import numpy as np
import pandas as pd
import nfn_crawring as nf
import gbq_service as gbq

In [8]:
df_krx = gbq.get_krx_list()
df_krx = df_krx.sort_values('market_cap', ascending=False).reset_index(drop=True)
df_krx

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,005930,삼성전자,1975-06-11,KOSPI,None,5969782550,444151821720000
1,000660,SK하이닉스,1996-12-26,KOSPI,None,728002365,98280319275000
2,373220,LG에너지솔루션,2022-01-27,KOSPI,None,234000000,88335000000000
3,207940,삼성바이오로직스,2016-11-10,KOSPI,None,71174000,56939200000000
4,005380,현대차,1974-06-28,KOSPI,None,211531506,41375562573600
...,...,...,...,...,...,...,...
2325,030790,비케이탑스,2010-11-02,KOSPI,None,21248145,19208323080
2326,071950,코아스,2005-08-04,KOSPI,None,30907210,15824491520
2327,078860,아이오케이,2006-06-27,KOSDAQ,중견기업부,4806323,15788771055
2328,008500,일정실업,1994-08-12,KOSPI,None,1200000,15480000000


In [9]:
collection_list = df_krx[['stock_code', 'corp_name']].values[0:2]
# collection_list = df_krx[['stock_code', 'corp_name']].values

In [10]:
df_gbq = pd.DataFrame()
df_exception = pd.DataFrame(columns=['stock_code', 'corp_name', 'err_msg'])

for index, corp in enumerate(collection_list):
    try:
        print('No. ', index)
        print(corp, '재무제표 데이터 크롤링 시작 ===>')
        # 연간 손익계산서
        df_is = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.IS, period=nf.Period.YEAR)
        df_is = gbq.convert_gbq_year(df_is, corp[0], corp[1], 'IS')
        
        # 연간 재무상태표
        df_bs = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.BS, period=nf.Period.YEAR)
        df_bs = gbq.convert_gbq_year(df_bs, corp[0], corp[1], 'BS')
        
        # 연간 현금흐름표
        df_cf = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.CF, period=nf.Period.YEAR)
        df_cf = gbq.convert_gbq_year(df_cf, corp[0], corp[1], 'CF')
        
        # 데이터 합치기
        df_gbq = pd.concat([df_gbq, df_is, df_bs, df_cf])
        
        print(corp, '재무제표 데이터 크롤링 종료 ===>\n\n')
        
    except Exception as e:
        err_msg = str(type(e)) + ': ' + str(e)
        print(err_msg)
        df_exception.loc[len(df_exception)] = [corp[0], corp[1], err_msg]

No.  0
['005930' '삼성전자'] 재무제표 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=0&finGubun=MAIN&frqTyp=0&cn=&encparam=Z2IvMmQvdk5FUTVsQVpOcGNZMkFSUT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=1&finGubun=MAIN&frqTyp=0&cn=&encparam=dnRJd3pzdlk3SmJkeTFVUEx4OUVNdz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=0&rpt=2&finGubun=MAIN&frqTyp=0&cn=&encparam=UG9NUzJPbVA1UmpwTzd4cGlxNFUxdz09
['005930' '삼성전자'] 재무제표 데이터 크롤링 종료 ===>


No.  1
['000660' 'SK하이닉스'] 재무제표 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=0&finGubun=MAIN&frqTyp=0&cn=&encparam=MUVMZTRyRDdtSjZRUEM5UEFpcmtJQT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=1&finGubun=MAIN&frqTyp=0&cn=&encparam=WVJKcmo2cW0zRHlSelA5VitFTE1NUT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=0&rpt=2&finGubun=MAIN&fr

In [11]:
df_gbq.to_csv('fs_year_240201.csv')

In [12]:
gbq.load_to_fs_year(df_gbq)

LoadJob<project=hello-phase3, location=asia-northeast3, id=46d4c79f-333b-4cb0-bed1-cb38699b48d6> 
Uploaded:  8070
